# PSNR

In [4]:
import pandas as pd
from pathlib import Path

background_map = {
    'The city of London': 'The city of London',
    'The Parthenon in front of the Great Pyramid': 'The Parthenon in front of the Great Pyramid',
    'A single beam of light enters the room from the ceiling The beam of light is illuminating an easel On the easel there is a Rembrandt painting of a raccoon': 'A single beam of light enters the room from the ceiling. The beam of light is illuminating an easel. On the easel, there is a Rembrandt painting of a raccoon.',
    'A sunset': 'A sunset',
    'Photograph of a wall along a city street with a watercolor mural of foes in a jazz band': 'Photograph of a wall along a city street with a watercolor mural of foxes in a jazz band.'
}

clothes_map = {
    'A_scientist': 'A scientist',
    'A_photograph_of_a_knight_in_shining_armor_holding_a_basketball': 'A photograph of a knight in shining armor holding a basketball',
    'The_Mona_Lisa': 'The Mona Lisa',
    'Salvador_Dalí': 'Salvador Dalí',
    'A_person_with_arms_like_a_tree_branch': 'A person with arms like a tree branch'
}

def parse_threshold(th_str: str) -> float:
    return int(th_str[2:]) / 10.0

rows = []
for file in sorted(Path('experiment').glob('*.jpg')):
    name = file.name
    parts = name[:-4].split('__')
    img_name = parts[0]
    background_raw = parts[2]
    th_raw = parts[3]
    clothing_raw = parts[4]
    row = {
        'nome_arquivo': name,
        'imagem': img_name,
        'threshold': parse_threshold(th_raw),
        'prompt_fundo': background_map.get(background_raw, background_raw),
        'prompt_roupa': clothes_map.get(clothing_raw, clothing_raw.replace('_', ' '))
    }
    rows.append(row)

df = pd.DataFrame(rows)


In [5]:
# -*- coding: utf-8 -*-
"""
PSNR por imagem (duas colunas): psnr_fundo e psnr_inpainting.
- Muito mais rápido que FID (não usa rede neural).
- Se houver múltiplas referências por prompt, tira a MÉDIA dos PSNRs.
- Opcional: filtrar por threshold (ex.: 1.3).

Saída: acrescenta colunas 'psnr_fundo' e 'psnr_inpainting' em experiment_metrics.csv.
"""

from __future__ import annotations
import os, math, unicodedata
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
import regex as re

import numpy as np
import pandas as pd
from PIL import Image

# =========================
# CONFIG
# =========================
EXPERIMENT_DIR = Path("experiment")
CSV_IN = None               # None = usar df do ambiente; ou "experiment.csv"
CSV_OUT = "experiment_metrics.csv"
RESIZE_TO = None            # ex.: (512, 512) para padronizar e acelerar; None = manter tamanho do gerado
TARGET_THRESHOLD = None     # ex.: 1.3 para filtrar; None = não filtra

# Mapeamento: prompt -> caminho(s) de referência
REFERENCE_IMAGES: Dict[str, Union[str, List[str]]] = {
    'Photograph of a wall along a city street with a watercolor mural of foxes in a jazz band.': 'reference_images/fox_mural.png',
    'The city of London': 'reference_images/london.png',
    'The Parthenon in front of the Great Pyramid': 'reference_images/partenon_great_pyramid.png',
    'A sunset': 'reference_images/sunset.png',
    'A single beam of light enters the room from the ceiling. The beam of light is illuminating an easel. On the easel, there is a Rembrandt painting of a raccoon.': 'reference_images/racoon.png',
    'A scientist': 'reference_images/scientist.png',
    'A photograph of a knight in shining armor holding a basketball': 'reference_images/knight_basketball.png',
    'The Mona Lisa': 'reference_images/monalisa.jpg',
    'Salvador Dalí': 'reference_images/salvador_dali.jpeg',
    'A person with arms like a tree branch': 'reference_images/tree_arms.png',
}

# =========================
# Helpers
# =========================
def _expand_paths(entry: Union[str, List[str]]) -> List[Path]:
    """Aceita string ou lista; inclui *_2.ext se existir."""
    entries = [entry] if isinstance(entry, str) else list(entry)
    out: List[Path] = []
    seen = set()
    for e in entries:
        p = Path(e)
        for cand in (p, p.with_name(f"{p.stem}_2{p.suffix}")):
            if cand.exists():
                key = str(cand.resolve()) if cand.exists() else str(cand)
                if key not in seen:
                    out.append(cand)
                    seen.add(key)
    return out

_punct = re.compile(r"[^\w\s]", flags=re.UNICODE)
def _norm(s: str) -> str:
    s = str(s).replace("_", " ")
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = _punct.sub(" ", s).lower().strip()
    s = " ".join(s.split())
    return s

def build_ref_map(mapping: Dict[str, Union[str, List[str]]]) -> Dict[str, List[Path]]:
    out: Dict[str, List[Path]] = {}
    for k, v in mapping.items():
        out[_norm(k)] = _expand_paths(v)
    return out

def find_refs(prompt: str, ref_map: Dict[str, List[Path]]) -> List[Path]:
    key = _norm(prompt)
    # exato
    if key in ref_map and ref_map[key]:
        return ref_map[key]
    # substring
    for k, paths in ref_map.items():
        if paths and (key in k or k in key):
            return paths
    return []

def psnr(img_a: np.ndarray, img_b: np.ndarray, data_range: float = 255.0) -> float:
    """PSNR em dB (quanto MAIOR, melhor). img_*: uint8 ou float32 [0..255]."""
    a = img_a.astype(np.float32)
    b = img_b.astype(np.float32)
    mse = np.mean((a - b) ** 2)
    if mse <= 1e-12:
        return float('inf')
    return 20.0 * math.log10(data_range / math.sqrt(mse))

def load_img(path: Path, resize_to: Optional[Tuple[int, int]]) -> np.ndarray:
    im = Image.open(path).convert("RGB")
    if resize_to is not None:
        im = im.resize(resize_to, Image.BILINEAR)
    return np.array(im, dtype=np.uint8)

def parse_threshold_from_name(name: str) -> Optional[float]:
    s = str(name)
    m = re.search(r'__th([0-9]+(?:\.[0-9]+)?)__', s, flags=re.IGNORECASE) \
        or re.search(r'_th([0-9]+(?:\.[0-9]+)?)_', s, flags=re.IGNORECASE) \
        or re.search(r'th([0-9]+(?:\.[0-9]+)?)', s, flags=re.IGNORECASE)
    if m:
        try: return float(m.group(1))
        except: pass
    m2 = re.search(r'__th([0-9]{1,3})__', s, flags=re.IGNORECASE) \
         or re.search(r'_th([0-9]{1,3})_', s, flags=re.IGNORECASE)
    if m2:
        d = m2.group(1)
        if d.isdigit():
            if len(d) == 1: return float(d)/10.0
            if len(d) == 2: return float(f"{d[0]}.{d[1]}")
            if len(d) == 3: return float(f"{d[0]}.{d[1:3]}")
            return float(d)
    return None

# =========================
# Carrega df
# =========================
if CSV_IN is None:
    if 'df' in globals():
        df_base = df.copy()
    elif 'df_out' in globals():
        df_base = df_out.copy()
    else:
        raise FileNotFoundError("Nem 'df' nem 'df_out' estão no ambiente. Defina CSV_IN para ler de arquivo.")
else:
    df_base = pd.read_csv(CSV_IN)

req = {"nome_arquivo", "prompt_roupa", "prompt_fundo"}
miss = req - set(df_base.columns)
if miss:
    raise ValueError(f"DataFrame não contém: {miss}")

# Extrai threshold (se quiser filtrar)
if TARGET_THRESHOLD is not None:
    if "threshold" in df_base.columns:
        th_series = pd.to_numeric(df_base["threshold"], errors="coerce")
    else:
        th_series = df_base["nome_arquivo"].map(parse_threshold_from_name)
    df_base = df_base.assign(_th_val=th_series)
    before = len(df_base)
    df_base = df_base[np.isfinite(df_base["_th_val"]) & (np.isclose(df_base["_th_val"], float(TARGET_THRESHOLD)))]
    print(f"[INFO] Filtradas {len(df_base)}/{before} linhas para threshold={TARGET_THRESHOLD}")

# =========================
# Loop PSNR
# =========================
ref_map = build_ref_map(REFERENCE_IMAGES)
psnr_fundo, psnr_inp = [], []

print(f"[INFO] Processando {len(df_base)} imagens para PSNR...")
for i, row in df_base.iterrows():
    img_path = EXPERIMENT_DIR / str(row["nome_arquivo"])
    if not img_path.exists():
        print(f"[AVISO] ausente: {img_path}")
        psnr_fundo.append(np.nan)
        psnr_inp.append(np.nan)
        continue

    # carrega imagem gerada
    img_arr = load_img(img_path, RESIZE_TO)

    # refs fundo
    fundo_refs = find_refs(str(row["prompt_fundo"]), ref_map)
    if not fundo_refs:
        print(f"[AVISO] sem ref fundo para: {row['prompt_fundo']}")
        psnr_fundo.append(np.nan)
    else:
        vals = []
        for rp in fundo_refs:
            ref_arr = load_img(rp, RESIZE_TO if RESIZE_TO else img_arr.shape[1::-1])
            # ajusta tamanho da ref ao da gerada, se necessário
            if ref_arr.shape != img_arr.shape:
                pil_ref = Image.fromarray(ref_arr)
                pil_ref = pil_ref.resize((img_arr.shape[1], img_arr.shape[0]), Image.BILINEAR)
                ref_arr = np.array(pil_ref, dtype=np.uint8)
            vals.append(psnr(img_arr, ref_arr))
        psnr_fundo.append(float(np.mean(vals)))

    # refs inpainting
    inp_refs = find_refs(str(row["prompt_roupa"]), ref_map)
    if not inp_refs:
        print(f"[AVISO] sem ref inpainting para: {row['prompt_roupa']}")
        psnr_inp.append(np.nan)
    else:
        vals = []
        for rp in inp_refs:
            ref_arr = load_img(rp, RESIZE_TO if RESIZE_TO else img_arr.shape[1::-1])
            if ref_arr.shape != img_arr.shape:
                pil_ref = Image.fromarray(ref_arr)
                pil_ref = pil_ref.resize((img_arr.shape[1], img_arr.shape[0]), Image.BILINEAR)
                ref_arr = np.array(pil_ref, dtype=np.uint8)
            vals.append(psnr(img_arr, ref_arr))
        psnr_inp.append(float(np.mean(vals)))

    if (len(psnr_fundo) % max(1, len(df_base)//20)) == 0:
        print(f"[INFO] {len(psnr_fundo)}/{len(df_base)} concluídas...")

# =========================
# Salva
# =========================
df_out_psnr = df_base.copy()
df_out_psnr["psnr_fundo"] = psnr_fundo
df_out_psnr["psnr_inpainting"] = psnr_inp

# Se já existe um experiment_metrics.csv com outras métricas, fazemos merge por nome_arquivo
try:
    if Path(CSV_OUT).exists():
        old = pd.read_csv(CSV_OUT)
        merged = old.merge(
            df_out_psnr[["nome_arquivo", "psnr_fundo", "psnr_inpainting"]],
            on="nome_arquivo",
            how="left"
        )
        merged.to_csv(CSV_OUT, index=False)
        print(f"[FINALIZADO] PSNR salvo/mesclado em: {CSV_OUT}")
    else:
        df_out_psnr.to_csv(CSV_OUT, index=False)
        print(f"[FINALIZADO] PSNR salvo em: {CSV_OUT}")
except Exception as e:
    df_out_psnr.to_csv(CSV_OUT, index=False)
    print(f"[FINALIZADO] PSNR salvo (modo fallback) em: {CSV_OUT}. Motivo do fallback: {e}")


[INFO] Processando 6000 imagens para PSNR...
[INFO] 300/6000 concluídas...
[INFO] 600/6000 concluídas...
[INFO] 900/6000 concluídas...
[INFO] 1200/6000 concluídas...
[INFO] 1500/6000 concluídas...
[INFO] 1800/6000 concluídas...
[INFO] 2100/6000 concluídas...
[INFO] 2400/6000 concluídas...
[INFO] 2700/6000 concluídas...
[INFO] 3000/6000 concluídas...
[INFO] 3300/6000 concluídas...
[INFO] 3600/6000 concluídas...
[INFO] 3900/6000 concluídas...
[INFO] 4200/6000 concluídas...
[INFO] 4500/6000 concluídas...
[INFO] 4800/6000 concluídas...
[INFO] 5100/6000 concluídas...
[INFO] 5400/6000 concluídas...
[INFO] 5700/6000 concluídas...
[INFO] 6000/6000 concluídas...
[FINALIZADO] PSNR salvo/mesclado em: experiment_metrics.csv


# SSIM

In [9]:
# -*- coding: utf-8 -*-
"""
SSIM por imagem (duas colunas): ssim_fundo e ssim_inpainting.
- Muito rápido (apenas operações por pixel/janela), sem redes neurais.
- Se houver múltiplas referências por prompt, tira a MÉDIA dos SSIMs.
- Opcional: filtrar por threshold (ex.: 1.3).

Saída: acrescenta colunas 'ssim_fundo' e 'ssim_inpainting' em experiment_metrics.csv.
"""

from __future__ import annotations
import os, unicodedata
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
import regex as re

import numpy as np
import pandas as pd
from PIL import Image

# =========================
# CONFIG
# =========================
EXPERIMENT_DIR = Path("experiment")
CSV_IN = None               # None = usar df/df_out do ambiente; ou ex.: "experiment.csv"
CSV_OUT = "experiment_metrics.csv"
RESIZE_TO = None            # ex.: (512, 512) para padronizar; None = manter tamanho do gerado
TARGET_THRESHOLD = None     # ex.: 1.3 para filtrar; None = não filtra

# Mapeamento: prompt -> caminho(s) de referência
REFERENCE_IMAGES: Dict[str, Union[str, List[str]]] = {
    'Photograph of a wall along a city street with a watercolor mural of foxes in a jazz band.': 'reference_images/fox_mural.png',
    'The city of London': 'reference_images/london.png',
    'The Parthenon in front of the Great Pyramid': 'reference_images/partenon_great_pyramid.png',
    'A sunset': 'reference_images/sunset.png',
    'A single beam of light enters the room from the ceiling. The beam of light is illuminating an easel. On the easel, there is a Rembrandt painting of a raccoon.': 'reference_images/racoon.png',
    'A scientist': 'reference_images/scientist.png',
    'A photograph of a knight in shining armor holding a basketball': 'reference_images/knight_basketball.png',
    'The Mona Lisa': 'reference_images/monalisa.jpg',
    'Salvador Dalí': 'reference_images/salvador_dali.jpeg',
    'A person with arms like a tree branch': 'reference_images/tree_arms.png',
}

# =========================
# Helpers
# =========================
def _expand_paths(entry: Union[str, List[str]]) -> List[Path]:
    """Aceita string ou lista; inclui *_2.ext se existir."""
    entries = [entry] if isinstance(entry, str) else list(entry)
    out: List[Path] = []
    seen = set()
    for e in entries:
        p = Path(e)
        for cand in (p, p.with_name(f"{p.stem}_2{p.suffix}")):
            if cand.exists():
                key = str(cand.resolve()) if cand.exists() else str(cand)
                if key not in seen:
                    out.append(cand)
                    seen.add(key)
    return out

_punct = re.compile(r"[^\w\s]", flags=re.UNICODE)
def _norm(s: str) -> str:
    s = str(s).replace("_", " ")
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = _punct.sub(" ", s).lower().strip()
    s = " ".join(s.split())
    return s

def build_ref_map(mapping: Dict[str, Union[str, List[str]]]) -> Dict[str, List[Path]]:
    out: Dict[str, List[Path]] = {}
    for k, v in mapping.items():
        out[_norm(k)] = _expand_paths(v)
    return out

def find_refs(prompt: str, ref_map: Dict[str, List[Path]]) -> List[Path]:
    key = _norm(prompt)
    # exato
    if key in ref_map and ref_map[key]:
        return ref_map[key]
    # substring
    for k, paths in ref_map.items():
        if paths and (key in k or k in key):
            return paths
    return []

def load_img(path: Path, resize_to: Optional[Tuple[int, int]]) -> np.ndarray:
    im = Image.open(path).convert("RGB")
    if resize_to is not None:
        im = im.resize(resize_to, Image.BILINEAR)
    return np.array(im, dtype=np.uint8)

def ssim_img(img_a: np.ndarray, img_b: np.ndarray) -> float:
    """SSIM ∈ [0,1], maior é melhor. img_* devem estar em uint8 [0..255]."""
    from skimage.metrics import structural_similarity as ssim
    # Ajuste de shape/tamanho
    if img_a.shape != img_b.shape:
        pil_b = Image.fromarray(img_b)
        pil_b = pil_b.resize((img_a.shape[1], img_a.shape[0]), Image.BILINEAR)
        img_b = np.array(pil_b, dtype=np.uint8)
    # Compatibilidade com versões de skimage
    try:
        val = ssim(img_a, img_b, data_range=255, channel_axis=-1,
                   gaussian_weights=True, sigma=1.5, use_sample_covariance=False)
    except TypeError:
        # versões antigas
        val = ssim(img_a, img_b, data_range=255, multichannel=True,
                   gaussian_weights=True, sigma=1.5, use_sample_covariance=False)
    return float(val)

def parse_threshold_from_name(name: str) -> Optional[float]:
    s = str(name)
    m = re.search(r'__th([0-9]+(?:\.[0-9]+)?)__', s, flags=re.IGNORECASE) \
        or re.search(r'_th([0-9]+(?:\.[0-9]+)?)_', s, flags=re.IGNORECASE) \
        or re.search(r'th([0-9]+(?:\.[0-9]+)?)', s, flags=re.IGNORECASE)
    if m:
        try: return float(m.group(1))
        except: pass
    m2 = re.search(r'__th([0-9]{1,3})__', s, flags=re.IGNORECASE) \
         or re.search(r'_th([0-9]{1,3})_', s, flags=re.IGNORECASE)
    if m2:
        d = m2.group(1)
        if d.isdigit():
            if len(d) == 1: return float(d)/10.0
            if len(d) == 2: return float(f"{d[0]}.{d[1]}")
            if len(d) == 3: return float(f"{d[0]}.{d[1:3]}")
            return float(d)
    return None

# =========================
# Carrega df
# =========================
if CSV_IN is None:
    if 'df' in globals():
        df_base = df.copy()
    elif 'df_out' in globals():
        df_base = df_out.copy()
    else:
        raise FileNotFoundError("Nem 'df' nem 'df_out' estão no ambiente. Defina CSV_IN para ler de arquivo.")
else:
    df_base = pd.read_csv(CSV_IN)

req = {"nome_arquivo", "prompt_roupa", "prompt_fundo"}
miss = req - set(df_base.columns)
if miss:
    raise ValueError(f"DataFrame não contém: {miss}")

# Extrai threshold (se quiser filtrar)
if TARGET_THRESHOLD is not None:
    if "threshold" in df_base.columns:
        th_series = pd.to_numeric(df_base["threshold"], errors="coerce")
    else:
        th_series = df_base["nome_arquivo"].map(parse_threshold_from_name)
    df_base = df_base.assign(_th_val=th_series)
    before = len(df_base)
    df_base = df_base[np.isfinite(df_base["_th_val"]) & (np.isclose(df_base["_th_val"], float(TARGET_THRESHOLD)))]
    print(f"[INFO] Filtradas {len(df_base)}/{before} linhas para threshold={TARGET_THRESHOLD}")

# =========================
# Loop SSIM
# =========================
ref_map = build_ref_map(REFERENCE_IMAGES)
ssim_fundo, ssim_inp = [], []

print(f"[INFO] Processando {len(df_base)} imagens para SSIM...")
for i, row in df_base.iterrows():
    img_path = EXPERIMENT_DIR / str(row["nome_arquivo"])
    if not img_path.exists():
        print(f"[AVISO] ausente: {img_path}")
        ssim_fundo.append(np.nan)
        ssim_inp.append(np.nan)
        continue

    # carrega imagem gerada (padrão: tamanho original)
    img_arr = load_img(img_path, RESIZE_TO)

    # refs fundo
    fundo_refs = find_refs(str(row["prompt_fundo"]), ref_map)
    if not fundo_refs:
        print(f"[AVISO] sem ref fundo para: {row['prompt_fundo']}")
        ssim_fundo.append(np.nan)
    else:
        vals = []
        for rp in fundo_refs:
            ref_arr = load_img(rp, RESIZE_TO if RESIZE_TO else img_arr.shape[1::-1])
            vals.append(ssim_img(img_arr, ref_arr))
        ssim_fundo.append(float(np.mean(vals)))

    # refs inpainting
    inp_refs = find_refs(str(row["prompt_roupa"]), ref_map)
    if not inp_refs:
        print(f"[AVISO] sem ref inpainting para: {row['prompt_roupa']}")
        ssim_inp.append(np.nan)
    else:
        vals = []
        for rp in inp_refs:
            ref_arr = load_img(rp, RESIZE_TO if RESIZE_TO else img_arr.shape[1::-1])
            vals.append(ssim_img(img_arr, ref_arr))
        ssim_inp.append(float(np.mean(vals)))

    if (len(ssim_fundo) % max(1, len(df_base)//20)) == 0:
        print(f"[INFO] {len(ssim_fundo)}/{len(df_base)} concluídas...")

# =========================
# Salva
# =========================
df_out_ssim = df_base.copy()
df_out_ssim["ssim_fundo"] = ssim_fundo
df_out_ssim["ssim_inpainting"] = ssim_inp

# Se já existe um experiment_metrics.csv com outras métricas, fazemos merge por nome_arquivo
try:
    if Path(CSV_OUT).exists():
        old = pd.read_csv(CSV_OUT)
        merged = old.merge(
            df_out_ssim[["nome_arquivo", "ssim_fundo", "ssim_inpainting"]],
            on="nome_arquivo",
            how="left"
        )
        merged.to_csv(CSV_OUT, index=False)
        print(f"[FINALIZADO] SSIM salvo/mesclado em: {CSV_OUT}")
    else:
        df_out_ssim.to_csv(CSV_OUT, index=False)
        print(f"[FINALIZADO] SSIM salvo em: {CSV_OUT}")
except Exception as e:
    df_out_ssim.to_csv(CSV_OUT, index=False)
    print(f"[FINALIZADO] SSIM salvo (modo fallback) em: {CSV_OUT}. Motivo do fallback: {e}")


[INFO] Processando 6000 imagens para SSIM...
[INFO] 300/6000 concluídas...
[INFO] 600/6000 concluídas...



KeyboardInterrupt

